In [1]:
import ee
import geemap
import geemap.eefolium as emap 

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/0gEMyJeH6yGeUz2sYPjxzRgcB5UVlhl7bKZ_vF-kQlUpezTc8zwGXXw

Successfully saved authorization token.


In [3]:
#Variabel
start = input("Tanggal Awal:")
finish = input("Tanggal Akhir:")
bounds = ee.Geometry.Rectangle(110, -6.4, 112, -7.78)

Tanggal Awal:2018-01-01
Tanggal Akhir:2020-06-05


In [4]:
#Image Colection1
imagecol = ee.ImageCollection("COPERNICUS/S2").filterDate(start, finish).filterBounds(bounds).filter('CLOUDY_PIXEL_PERCENTAGE < 5').median()

In [59]:
#Image Colection2
imagecol = ee.ImageCollection("COPERNICUS/S2").filterDate(start, finish).filterBounds(bounds).filter('CLOUDY_PIXEL_PERCENTAGE < 5').median()
imagecol1 = ee.ImageCollection("COPERNICUS/S2").filterDate('2013-01-01', '2017-12-30').filterBounds(bounds).filter('CLOUDY_PIXEL_PERCENTAGE < 5').median()

In [15]:
#Input ROI
roi = geemap.shp_to_ee(input("Lokasi shp:"))

Lokasi shp:D:\Cloud Computing\SHP GUNUNG KIDUL\BATAS_ADMINISTRASI.shp


In [18]:
#Raster Calculator
NDVI = imagecol.expression('(NIR - RED)/(NIR + RED)',
                          {'NIR': imagecol.select('B8'),
                          'RED': imagecol.select('B4')})

In [60]:
#Raster Calculator
NDVI1 = imagecol1.expression('(NIR - RED)/(NIR + RED)',
                             {'NIR': imagecol1.select('B8'),
                              'RED': imagecol1.select('B4')})

In [36]:
#Raster Calculator
leftlayer = geemap.ee_tile_layer(imagecol.clip(roi),
                                 {'bands':['B4', 'B3','B2'],
                                  'min':0, 'max':2500},
                                 name='ImageCollection')
rightlayer = geemap.ee_tile_layer(NDVI.clip(roi),
                                  {'palette': ['red', 'yellow', 'green']},
                                  name='NDVI')

In [90]:
#Visualisasi Layer
Map = geemap.Map()
Map.set_center(110.64, -7.91, zoom=12)
Map.addLayer(imagecol.clip(roi),
             {'bands':['B4', 'B3','B2'], 'min':0,'max':2500}, name='ImageCollection')
Map.addLayer(NDVI.clip(roi),
            {'palette': ['red', 'yellow', 'green']},
            name='NDVI')
Map

Map(center=[-7.91, 110.64], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

In [65]:
#Split Variable
NDVI1tile = geemap.ee_tile_layer(NDVI1.clip(roi),
                                 {'palette': ['red', 'yellow', 'green']},
                                 name='NDVI1')
NDVI2tile = geemap.ee_tile_layer(NDVI.clip(roi),
                                 {'palette': ['red', 'yellow', 'green']},
                                 name='NDVI2')

In [66]:
#Visualisasi Split
Map = geemap.Map()
Map.centerObject(roi, zoom=12)

Map.split_map(NDVI1tile, NDVI2tile)

Map

Map(center=[-7.994035739567542, 110.61208091551455], controls=(WidgetControl(options=['position'], widget=HBox…

In [71]:
#Visualisasi TimeSeries
sentinel_ts = geemap.sentinel2_timeseries(roi=roi, start_year=2015, end_year=2020, start_date='01-01', end_date='12-30')

In [82]:
layer_names = ['Sentinel2_' + str(year) for year in range(2015,2021)]
print(layer_names)

['Sentinel2_2015', 'Sentinel2_2016', 'Sentinel2_2017', 'Sentinel2_2018', 'Sentinel2_2019', 'Sentinel2_2020']


In [85]:
sentinel_vis = {'bands':['Red', 'Green','Blue'], 'min':0,'max':2500}

In [86]:
Map = geemap.Map()
Map.centerObject(roi, zoom=12)
Map.ts_inspector(left_ts=sentinel_ts, right_ts=sentinel_ts, 
                 left_names=layer_names, right_names=layer_names,
                 left_vis=sentinel_vis, right_vis=sentinel_vis)
Map

Map(center=[-7.994035739567542, 110.61208091551455], controls=(WidgetControl(options=['position'], widget=Drop…

In [48]:
#Draw Features
feature = Map.draw_features
polygon = ee.Geometry.MultiPolygon(feature)

In [34]:
#Export Image
geemap.ee_export_image(NDVI,
                      filename='NDVI.tif', scale=10,
                      region=feature.geometry())

Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'D:\\Cloud Computing\\NDVI.zip'


In [91]:
!git clone https://github.com/SandiRizqi/Palms-Crown-Detection-and-Health-Assessment-with-Python-Arcpy.git

Cloning into 'Palms-Crown-Detection-and-Health-Assessment-with-Python-Arcpy'...
